# Cohere Reranker

CohereRerank는 검색 결과의 관련성을 재순위하는 기능을 제공하는 도구입니다.    
이는 특히 대량의 문서나 데이터에서 특정 쿼리에 대한 가장 관련성 높은 결과를 찾아내는 데 유용합니다. 

CohereRerank의 주요 특장점은 다음과 같습니다:
- 검색 결과의 관련성 향상:   
CohereRerank는 사용자 쿼리와 문서 간의 관련성을 평가하여, 가장 관련성 높은 문서를 상위에 배치합니다.    
이는 전통적인 키워드 기반 검색 시스템에서 발생할 수 있는 불필요한 결과를 줄이고, 사용자가 원하는 정보를 더 빠르게 찾을 수 있게 해줍니다

- 다국어 지원:   
CohereRerank는 100개 이상의 언어에 대한 검색 품질을 보장합니다. 이는 다양한 언어로 된 문서를 처리할 때 매우 유용하며, 글로벌 사용자 기반을 가진 애플리케이션에 특히 적합합니다

- 대규모 문서 처리 능력:   
CohereRerank는 최대 1,000개의 문서를 재순위할 수 있으며, 이는 지식 베이스나 대규모 데이터 세트에 적용할 수 있음을 의미합니다

#### 참고문서

- https://txt.cohere.com/rerank/

- https://docs.llamaindex.ai/en/stable/examples/node_postprocessor/CohereRerank/?h=cohere

In [1]:
%pip install llama-index-postprocessor-cohere-rerank

Note: you may need to restart the kernel to use updated packages.


#### Download Data

In [2]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2024-04-04 13:16:26--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
raw.githubusercontent.com (raw.githubusercontent.com) 해석 중... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
다음으로 연결 중: raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 75042 (73K) [text/plain]
저장 위치: `data/paul_graham/paul_graham_essay.txt'

data/paul_graham/pa 100%[===================>]  73.28K  --.-KB/s    /  0.04s   

2024-04-04 13:16:26 (1.63 MB/s) - `data/paul_graham/paul_graham_essay.txt' 저장함 [75042/75042]



In [5]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from dotenv import load_dotenv
load_dotenv()

True

In [6]:
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.core import Settings

embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.embed_model = embed_model

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [2]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader
)

# load documents
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

# build index
index = VectorStoreIndex.from_documents(documents=documents)

#### Set CohereRerank

- top_n: 상위 n개의 청크를 뽑아냄
- model: 최대 100개국의 다국어 지원(rerank-multilingual-v2.0)

In [13]:
from llama_index.postprocessor.cohere_rerank import CohereRerank

cohere_rerank = CohereRerank(
    top_n=3, 
    model="rerank-multilingual-v2.0",
    api_key="MDoHOxPWAanvMhXTqJlGdNkccofm5QZNdWUGRz4H"
)

#### 가장 관련성이 높은 상위 10개 노드를 검색한 다음 Cohere Rerank로 필터링합니다.

In [14]:
query_engine = index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[cohere_rerank],
)
response = query_engine.query(
    "What did Sam Altman do in this essay?",
)

TypeError: list indices must be integers or slices, not builtin_function_or_method

In [7]:
# from llama_index.core.response.pprint_utils import pprint_response, pprint_source_node, pprint_metadata, pprint
from llama_index.core.response.notebook_utils import display_response, display_source_node, display_metadata

display_response(response, source_length=500, show_source=True, show_source_metadata=True)

**`Final Response:`** Repeat: Sam Altman was recruited to take over as the president of Y Combinator after Paul Graham decided to step down from his role.

---

**`Source Node 1/10`**

**Node ID:** 6feed6f0-31ae-429c-938f-de6b7d2106e2<br>**Similarity:** 0.809807942240262<br>**Text:** Except for a few officially anointed thinkers who went to the right parties in New York, the only people allowed to publish essays were specialists writing about their specialties. There were so many essays that had never been written, because there had been no way to publish them. Now they could be, and I was going to write them. [12]

I've worked on several different things, but to the extent there was a turning point where I figured out what to work on, it was when I started publishing ess...<br>**Metadata:** {'file_path': '/Users/heewungsong/Experiment/Visa_Rag/study/llama-index/ReRanker/data/paul_graham/paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75042, 'creation_date': '2024-04-04', 'last_modified_date': '2024-04-04'}<br>

---

...

**`Source Node 9/10`**

**Node ID:** 8d0ca9ab-7627-4da7-bc2b-1957d3a469ad<br>**Similarity:** 0.7928642451235037<br>**Text:** It was not, in fact, simply a matter of teaching SHRDLU more words. That whole way of doing AI, with explicit data structures representing concepts, was not going to work. Its brokenness did, as so often happens, generate a lot of opportunities to write papers about various band-aids that could be applied to it, but it was never going to get us Mike.

So I looked around to see what I could salvage from the wreckage of my plans, and there was Lisp. I knew from experience that Lisp was interest...<br>**Metadata:** {'file_path': '/Users/heewungsong/Experiment/Visa_Rag/study/llama-index/ReRanker/data/paul_graham/paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75042, 'creation_date': '2024-04-04', 'last_modified_date': '2024-04-04'}<br>

---

**`Source Node 10/10`**

**Node ID:** a132ea42-bd78-49c3-ada0-46edb2bb9729<br>**Similarity:** 0.792675098405147<br>**Text:** They should make a larger number of smaller investments instead of a handful of giant ones, they should be funding younger, more technical founders instead of MBAs, they should let the founders remain as CEO, and so on.

One of my tricks for writing essays had always been to give talks. The prospect of having to stand up in front of a group of people and tell them something that won't waste their time is a great spur to the imagination. When the Harvard Computer Society, the undergrad compute...<br>**Metadata:** {'file_path': '/Users/heewungsong/Experiment/Visa_Rag/study/llama-index/ReRanker/data/paul_graham/paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75042, 'creation_date': '2024-04-04', 'last_modified_date': '2024-04-04'}<br>

#### Raw Engine 으로 테스트 (Without Cohere Rerank )

In [30]:
base_query_engine = index.as_query_engine(
    similarity_top_k=2,
)
base_response = base_query_engine.query(
    "What did Sam Altman do in this essay?",
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [31]:
display_response(base_response, source_length=500, show_source=True, show_source_metadata=True)

**`Final Response:`** Sam Altman was asked if he wanted to be the president of Y Combinator, initially declining the offer as he wanted to start a startup to make nuclear reactors. However, after persistent persuasion, he eventually agreed to take over as the president starting with the winter 2014 batch.

---

**`Source Node 1/2`**

**Node ID:** 9af3f019-6c27-4718-b76f-43b412d279d2<br>**Similarity:** 0.5949466309972675<br>**Text:** Except for a few officially anointed thinkers who went to the right parties in New York, the only people allowed to publish essays were specialists writing about their specialties. There were so many essays that had never been written, because there had been no way to publish them. Now they could be, and I was going to write them. [12]

I've worked on several different things, but to the extent there was a turning point where I figured out what to work on, it was when I started publishing ess...<br>**Metadata:** {'file_path': '/Users/heewungsong/Experiment/Visa_Rag/study/llama-index/ReRanker/data/paul_graham/paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75042, 'creation_date': '2024-04-04', 'last_modified_date': '2024-04-04'}<br>

---

**`Source Node 2/2`**

**Node ID:** f31edbc9-e8c5-4694-a7d7-7b90aa1f9f9b<br>**Similarity:** 0.5793073754036143<br>**Text:** One day in 2010, when he was visiting California for interviews, Robert Morris did something astonishing: he offered me unsolicited advice. I can only remember him doing that once before. One day at Viaweb, when I was bent over double from a kidney stone, he suggested that it would be a good idea for him to take me to the hospital. That was what it took for Rtm to offer unsolicited advice. So I remember his exact words very clearly. "You know," he said, "you should make sure Y Combinator isn'...<br>**Metadata:** {'file_path': '/Users/heewungsong/Experiment/Visa_Rag/study/llama-index/ReRanker/data/paul_graham/paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75042, 'creation_date': '2024-04-04', 'last_modified_date': '2024-04-04'}<br>

### E9 Visa Document Test

#### Load Vector Store from Chroma Vectore DB

In [32]:
import chromadb
from llama_index.core import VectorStoreIndex, load_index_from_storage
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.openai import OpenAIEmbedding

db = chromadb.PersistentClient(path="../Storing/chroma")

collection_name = "E9_VISA_GUIDE" # Guide

chroma_collection = db.get_collection(collection_name)
chroma_vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

embed_model = OpenAIEmbedding()
visa_guide_index = VectorStoreIndex.from_vector_store(vector_store=chroma_vector_store, embed_model=embed_model)

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


#### Load CohereRerank

In [70]:
from llama_index.postprocessor.cohere_rerank import CohereRerank

cohere_reranker = CohereRerank(top_n=3, model="rerank-multilingual-v2.0")

#### With Cohere 와 Without Cohere 비교

In [40]:
retriever = visa_guide_index.as_retriever(similarity_top_k=10)

In [64]:
def print_source_nodes(sources):
    for idx, source in enumerate(sources):
        print(f"Rank: {idx + 1}")
        print(f"Score: {source.score:.3f}")
        print(f"Page: {source.metadata['page_label']}")
        print(f"Title: {source.metadata['document_title']}")
        print(f"Text: {source.get_content()[:100]}")
        print(f"----------------------------------------------------------------\n")

In [65]:
"""
답: 비전문취업(E-9) 자격 사증발급인정서 발급절차
결과: 3순위
"""

query = "E-9 비자 신청 시 필요한 서류는 무엇인가요?"

source_nodes_without_cohere = retriever.retrieve(query)
print_source_nodes(source_nodes_without_cohere)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Rank: 1
Score: 0.785
Page: 6
Title: 고용허가제 해당자는 사증발급인정서를 받아야만 E-9 사증을 발급받을 수 있음
Text: 사증발급인정서를 받아야만 E-9 사증을 발급 받을수있음
        ● 비전문취업(E-9) 사증은 출입국ᆞ외국인청(사무소ᆞ출장소)장이 발급한 사증발급인정서에 의해서만 사증을 발급
----------------------------------------------------------------

Rank: 2
Score: 0.772
Page: 1
Title: E-9 Visa Guide: 목차
Text: # E-9: 고용허가제 비전문취업 비자

        고용허가제
            고용허가제 해당자와 활동범위
        고용허가제 해당자에 대한 사증 발급
       
----------------------------------------------------------------

Rank: 3
Score: 0.745
Page: 8
Title: 비전문취업(E-9) 자격 사증발급인정서 발급절차
Text: 비전문취업(E-9) 자격 사증발급인정서 발급절차
        ● 접수/신청 방법
            ○ 비전문취업자를 초청하고자 하는 사업장의 장이 사업장 주소지를 관할하는 출
----------------------------------------------------------------

Rank: 4
Score: 0.731
Page: 21
Title: 고용허가제 해당자의 고용변동 신고
Text: 고용허가제 해당자의 고용변동 신고
        ● 신고의무자
            ○ 비전문취업(E-9) 외국인

In [75]:
"""
답: 
    1. 고용허가제 농업 분야 외국인근로자의 근무처(직장) 추가
    2. 고용허가제 해당자가 근무처(직장)를 변경하는 절차 및 제출 서류
결과:
    1. 고용허가제 농업 분야 외국인근로자의 근무처(직장) 추가: 2순위
    2. 고용허가제 해당자가 근무처(직장)를 변경하는 절차 및 제출 서류: 10순위
"""

query = "저는 베트남에서 한국으로 취업을 희망하는 학생입니다. 한국의 농장에서 일하려고 하는데 준비해야하는 절차는 무엇이 있나요?"


source_nodes_without_cohere = retriever.retrieve(query)
print_source_nodes(source_nodes_without_cohere)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Rank: 1
Score: 0.687
Page: 2
Title: 고용허가제 법률적 개요와 고용허가제 선정국가
Text: 고용허가제 
            - ｢외국인근로자의 고용 등에 관한 법률｣에 의거, 사업주에게 외국인근로자의 고용을 허가하고, 외국인 근로자에게는 당해 사업주에게 고용되는 조건으
----------------------------------------------------------------

Rank: 2
Score: 0.685
Page: 16
Title: 고용허가제 농업 분야 외국인근로자의 근무처(직장) 추가
Text: 고용허가제 농업 분야 외국인근로자의 근무처(직장) 추가
        ● 계절적으로 업무량의 차이가 큰 일부 농업분야에 근무하는 외국인근로자가 원사업장과의 근로계약은 유지하면서(무
----------------------------------------------------------------

Rank: 3
Score: 0.678
Page: 19
Title: 고용허가제 해당자의 재입국허가
Text: 고용허가제 해당자의 재입국허가
        ● 재입국허가 면제 제도 시행(’10.12.1.자 개정 시행규칙)
            ○ 등록을 필한 외국인이 출국한 날부터 1년 이
----------------------------------------------------------------

Rank: 4
Score: 0.675
Page: 8
Title: 비전문취업(E-9) 자격 사증발급인정서 발급절차
Text: 비전문취업(E-9) 자격 사증발급인정서 발급절차
        ● 접수/신청 방법
            ○ 비전문취업자를 초청

In [67]:
"""
답: 고용허가제 해당자의 재입국특례 제도(구 성실근로자 제도)
결과: 1순위
"""

query = "재입국 특례 제도 신청을 위한 조건은 무엇인가요?"


source_nodes_without_cohere = retriever.retrieve(query)
print_source_nodes(source_nodes_without_cohere)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Rank: 1
Score: 0.765
Page: 9
Title: 재입국특례 제도(구 성실근로자 제도)
Text: 재입국특례 제도(구 성실근로자 제도)
        ● 재입국 특례 제도란
            ○ 일정한 요건을 갖춘 성실외국인 근로자가 취업활동 기간이 만료되어 출국하기 전에 
----------------------------------------------------------------

Rank: 2
Score: 0.736
Page: 19
Title: 고용허가제 해당자의 재입국허가
Text: 고용허가제 해당자의 재입국허가
        ● 재입국허가 면제 제도 시행(’10.12.1.자 개정 시행규칙)
            ○ 등록을 필한 외국인이 출국한 날부터 1년 이
----------------------------------------------------------------

Rank: 3
Score: 0.728
Page: 11
Title: 재입국특례자에 대한 우대 내용
Text: 재입국특례자에 대한 우대 내용
        ● 사업주
            ○ (내국인 구인노력 면제) 고용허가서 발급 시 사업주의 내국인 구인노력 불요
            ○
----------------------------------------------------------------

Rank: 4
Score: 0.720
Page: 10
Title: 재입국특례자에 대한 사증 신청 및 발급 방법
Text: 재입국특례자에 대한 사증 신청 및 발급 방법
        ● 사증 발급 방법
            ○ 출입국ᆞ외국인청(사무소ᆞ출장소)장이 발급한 사증발급인정서

#### With Cohere Reranker

In [71]:
"""
답: 비전문취업(E-9) 자격 사증발급인정서 발급절차
결과: 3순위 => 1순위로 변경됨
"""

query = "E-9 비자 신청 시 필요한 서류는 무엇인가요?"

source_nodes_without_cohere = retriever.retrieve(query)
source_nodes_with_cohere = cohere_reranker.postprocess_nodes(source_nodes_without_cohere, query_str=query)

print_source_nodes(source_nodes_with_cohere)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Rank: 1
Score: 0.997
Page: 8
Title: 비전문취업(E-9) 자격 사증발급인정서 발급절차
Text: 비전문취업(E-9) 자격 사증발급인정서 발급절차
        ● 접수/신청 방법
            ○ 비전문취업자를 초청하고자 하는 사업장의 장이 사업장 주소지를 관할하는 출
----------------------------------------------------------------

Rank: 2
Score: 0.911
Page: 1
Title: E-9 Visa Guide: 목차
Text: # E-9: 고용허가제 비전문취업 비자

        고용허가제
            고용허가제 해당자와 활동범위
        고용허가제 해당자에 대한 사증 발급
       
----------------------------------------------------------------

Rank: 3
Score: 0.771
Page: 5
Title: E-9 근로자를 채용할 수 있는 업종과 범위
Text: 사증발급 허용업종 및 체류자격 약호(기호): E-9 근로자를 채용할 수 있는 업종과 범위
        ● 제조업: 제조업, 뿌리산업
            ○ 체류자격: E-9-1
----------------------------------------------------------------



In [76]:
"""
답: 
    1. 고용허가제 농업 분야 외국인근로자의 근무처(직장) 추가
    2. 고용허가제 해당자가 근무처(직장)를 변경하는 절차 및 제출 서류
결과:
    1. 고용허가제 농업 분야 외국인근로자의 근무처(직장) 추가: 2순위 => 1순위
    2. 고용허가제 해당자가 근무처(직장)를 변경하는 절차 및 제출 서류: 10순위 => 2순위
"""

query = "저는 베트남에서 한국으로 취업을 희망하는 학생입니다. 한국의 농장에서 일하려고 하는데 준비해야하는 절차는 무엇이 있나요?"


source_nodes_without_cohere = retriever.retrieve(query)
source_nodes_with_cohere = cohere_reranker.postprocess_nodes(source_nodes_without_cohere, query_str=query)

print_source_nodes(source_nodes_with_cohere)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Rank: 1
Score: 0.040
Page: 16
Title: 고용허가제 농업 분야 외국인근로자의 근무처(직장) 추가
Text: 고용허가제 농업 분야 외국인근로자의 근무처(직장) 추가
        ● 계절적으로 업무량의 차이가 큰 일부 농업분야에 근무하는 외국인근로자가 원사업장과의 근로계약은 유지하면서(무
----------------------------------------------------------------

Rank: 2
Score: 0.022
Page: 15
Title: 고용허가제 해당자가 근무처(직장)를 변경하는 절차 및 제출 서류
Text: 고용허가제 해당자가 근무처(직장)를 변경하는 절차 및 제출 서류
        ● 근무처(직장) 변경 절차
            ○ 사용자와 근로계약 종료 후 1월 이내에 사업장 
----------------------------------------------------------------

Rank: 3
Score: 0.018
Page: 8
Title: 비전문취업(E-9) 자격 사증발급인정서 발급절차
Text: 비전문취업(E-9) 자격 사증발급인정서 발급절차
        ● 접수/신청 방법
            ○ 비전문취업자를 초청하고자 하는 사업장의 장이 사업장 주소지를 관할하는 출
----------------------------------------------------------------

